Note that the labelling of these sections is based on the version of the paper completed on 15/08/22. This cell should be updated in the event of a change to the labelling in the paper.

Currently running on Sagemath version 9.4.rc0, Release Date: 2021-07-27.
Using Python 3.9.5.

This notebook is intended to be ran sequentially from the beginning. Changing the order in which cells are ran may lead to errors. 

## Quotient by $(12)$ in the $\mathbb{P}^4$-model

In [1]:
# Same method as in previous notebooks
# Note the choice of transform matrix, which will make the subsequent quotient (34) clearer. 

P3s.<s1, s2, s3, s4> = QQ[]
v2s = matrix([[1,1,0,0],[1,-1, 0,0],[0,0,1,1],[0,0,1,-1]])
s2v = v2s.inverse()
v1, v2, v3, v4 = s2v*vector([s1, s2, s3, s4])

# We then define our two constraints (that is, we have already defined v5 := -v1-v2-v3-v4)
Q2 = v1^2+v2^2+v3^2+v4^2+(v1+v2+v3+v4)^2
Q3 = v1^3+v2^3+v3^3+v4^3-(v1+v2+v3+v4)^3

print("The defining equations in terms of the semi-invariants:")
print("Q2: ", Q2)
print("Q3: ", Q3)
print()

# Take invariants s=s1, t=s2^2, u=s3, v=s4
P3si.<s,t,u,v> = QQ[]
Q2 = 3/2*s^2 + 1/2*t + 2*s*u + 3/2*u^2 + 1/2*v^2
Q3 = -3/4*s^3 + 3/4*s*t - 3*s^2*u - 3*s*u^2 - 3/4*u^3 + 3/4*u*v^2

# In writing these, we notice we can eliminate t
tsub = -2*(3/2*s^2 + 2*s*u + 3/2*u^2 + 1/2*v^2)
poly = Q3(s,tsub,u,v)
# We extract the polynomial via the numerator 
P3sir.<s,u,v> = QQ[]
poly = P3sir(poly(s,0,u,v).numerator())
print("The defining equation in terms of the invariants:")
print("reduced polynomial:", poly.factor(), "\n")

# For the next cell it will be useful to see the curve in Weierstrass form
MP = Maple()
MP.with_package('algcurves')
wf_data = MP('Weierstrassform({}, u, v, x, y)'.format(poly))
poly_wp = wf_data.op(1)
print("Output of Weierstrassform: ")
print(poly_wp)

The defining equations in terms of the semi-invariants:
Q2:  3/2*s1^2 + 1/2*s2^2 + 2*s1*s3 + 3/2*s3^2 + 1/2*s4^2
Q3:  -3/4*s1^3 + 3/4*s1*s2^2 - 3*s1^2*s3 - 3*s1*s3^2 - 3/4*s3^3 + 3/4*s3*s4^2

The defining equation in terms of the invariants:
reduced polynomial: (-3) * (4*s^3 + 8*s^2*u + 7*s*u^2 + u^3 + s*v^2 - u*v^2) 

Output of Weierstrassform: 
x^3-25/3*s^4*x+2950/27*s^6+y^2


In [2]:
# We can then check the translation we get from the subsequent action of (345). 
load("riemann_surface.py")
from riemann_theta.riemann_theta import RiemannTheta

# We will use conventions of Mckean and Moll
a4 = -25/3
a6 = +2950/27
R.<x,y> = QQ[]
f5 = y^2-(x^3+a4*x+a6)
C5 = Curve(f5)
P = C5.places_at_infinity()[0]
integration_method = 'rigorous'
prec = 60
S5 = RiemannSurface(f5, prec=prec, integration_method=integration_method)
g = S5.genus
PM = S5.period_matrix()

t5 = S5.riemann_matrix()
# Let's verify that using the Weierstrass p would invert this naturally
# We use the results on pp. 132-133 of McKean and Moll
th = RiemannTheta(t5)
nw2 = th(char=[[1],[0],2])
nw3 = th(char=[[0],[0],2])
nw4 = th(char=[[0],[1],2])

# Let's verify some identities we should have for the theta function. 
if not (nw4^4-nw3^4+nw2^4).abs() < 1e-10:
    raise ValueError("check on theta funciton failed")
# This second identity is in Whittaker and Watson. Note there it is missing the pi.
# This is as the definition in WW excludes a factor of pi that we have in the sum. 
nw1p = -th(char=[[1],[1],2], derivs=[0])
if not ((nw1p-(S5._RR.pi()*nw2*nw3*nw4)).abs() < 1e-10):
    raise ValueError("check on theta funciton failed")

e1 = S5._RR.pi()^2*(nw4^4+nw3^4)/3
e2 = S5._RR.pi()^2*(nw2^4-nw4^4)/3
e3 = -S5._RR.pi()^2*(nw2^4+nw3^4)/3

if not ((e1+e2+e3).abs() < 1e-10):
    raise ValueError("check on theta funciton failed")

# Clearly this formula only works for wp defined wrt the lattice ZZ+tau*ZZ.
wp_normalised = lambda zi: (S5._RR.pi()*nw2*nw3*th(char=[[0],[1],2],z=(zi,))/th(char=[[1],[1],2],z=(zi,)))^2+e3
# We rectify through scaling 
mu = PM[0,0]
wp = lambda zi : wp_normalised(zi/mu)/mu^2
# Choose some arebitrary point in the lattice as a basepoint for this check
sz = CC(1/10+I/10)
wpz = wp(sz)
if ((wpz-wp(sz+PM[0,0])).abs() > 1e-10) or ((wpz-wp(sz+PM[0,1])).abs() > 1e-10):
    raise ValueError("check on theta funciton failed")
    
# Now we are confident we have a correct Weierstrass p function for the curve we
# we can check the translation that corresponds to (345) in this homology basis. 
# We give the base coordinate of the image of inf under the translation
inf_t_x = S5._CC(-25/3)

# We loop through all 1/3-lattice vectors and see if they are the give the same basepoint value
# (calculated via the Weierstrass p) as the image of infinity, would correspond to zero in the 
# lattice. 
for r in range(3):
    for s in range(3):
        ttx = wp((r*PM[0,0]+s*PM[0,1])/3)
        if (ttx - inf_t_x).abs() > 1e-10:
            continue
        print([r,s])
# This output tells us that the two possible trasnforms are z -> z + 1/3 or z -> z + 2/3
# This is what we expect, as we cannot distinguish between the two fibre vlaues of +-20 by this test,
# so one will correspond to (345) and the other to (354). 

[1, 0]
[2, 0]
